# Modelos de Ecuaciones Estructurales (SEM) - Variables Sumadas

Este notebook implementa **modelos de path analysis** para analizar la relación entre las variables ideológicas y las variables de cambio de opinión/tiempo **sumadas** (sin filtrar).

## Modelo para cada variable dependiente:

```
Indice_Progresismo ────→ Y
                          ↑
Indice_Conservadurismo ──→
```

**Ecuación:**
```
Y = β₀ + β₁(Indice_Progresismo) + β₂(Indice_Conservadurismo) + ε
```

## Variables Dependientes (8 modelos):

**Cambio de Opinión (CO):**
1. `Cambio_Op_Sum_Pro_Izq`
2. `Cambio_Op_Sum_Pro_Der`
3. `Cambio_Op_Sum_Con_Izq`
4. `Cambio_Op_Sum_Con_Der`

**Cambio de Tiempo (CT):**
5. `Cambio_Tiempo_Sum_Pro_Izq`
6. `Cambio_Tiempo_Sum_Pro_Der`
7. `Cambio_Tiempo_Sum_Con_Izq`
8. `Cambio_Tiempo_Sum_Con_Der`

## Métricas Reportadas:

- **Coeficientes β** (estandarizados y no estandarizados)
- **Error estándar**
- **Estadístico z**
- **p-valor**
- **R²** (varianza explicada)
- **AIC** (Akaike Information Criterion)
- **BIC** (Bayesian Information Criterion)

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Imports para SEM
try:
    from semopy import Model
    from semopy.inspector import inspect
    print("✓ semopy disponible")
    USAR_SEMOPY = True
except ImportError:
    print("⚠️  semopy no disponible, usando statsmodels OLS")
    from statsmodels.api import OLS, add_constant
    from sklearn.preprocessing import StandardScaler
    USAR_SEMOPY = False

from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

print("✓ Librerías cargadas exitosamente")

## 1. Cargar Datos

In [ ]:
# Rutas
Ruta_Base = os.path.join(os.getcwd(), '..', 'Data', 'Bases definitivas')
Excel_Generales = os.path.join(Ruta_Base, 'Generales.xlsx')
Excel_Ballotage = os.path.join(Ruta_Base, 'Ballotage.xlsx')

# Cargar DataFrames
df_Generales = pd.read_excel(Excel_Generales)
df_Ballotage = pd.read_excel(Excel_Ballotage)

dfs_Finales = {
    'Generales': df_Generales,
    'Ballotage': df_Ballotage
}

print(f"✓ Datos cargados:")
print(f"  - Generales: {len(df_Generales)} registros")
print(f"  - Ballotage: {len(df_Ballotage)} registros")

## 2. Definir Variables

In [ ]:
# Variables predictoras
Predictores = [
    'Indice_Progresismo',
    'Indice_Conservadurismo'
]

# Variables dependientes - Sumadas
Outcomes_Sumadas = [
    'Cambio_Op_Sum_Pro_Izq',
    'Cambio_Op_Sum_Pro_Der',
    'Cambio_Op_Sum_Con_Izq',
    'Cambio_Op_Sum_Con_Der',
    'Cambio_Tiempo_Sum_Pro_Izq',
    'Cambio_Tiempo_Sum_Pro_Der',
    'Cambio_Tiempo_Sum_Con_Izq',
    'Cambio_Tiempo_Sum_Con_Der'
]

print(f"✓ Variables definidas:")
print(f"  - Predictores: {len(Predictores)}")
print(f"  - Outcomes: {len(Outcomes_Sumadas)}")

## 3. Función para Ejecutar Modelo SEM

In [ ]:
def Ejecutar_Modelo_Path(df, outcome, predictores, nombre_modelo):
    """
    Ejecuta un modelo de path analysis (regresión múltiple).
    
    Parámetros:
    -----------
    df : DataFrame
        Datos
    outcome : str
        Variable dependiente
    predictores : list
        Lista de predictores
    nombre_modelo : str
        Nombre del modelo (para reportes)
    
    Retorna:
    --------
    dict : Resultados del modelo
    """
    
    # Verificar que las variables existan
    variables_necesarias = [outcome] + predictores
    variables_existentes = [v for v in variables_necesarias if v in df.columns]
    
    if len(variables_existentes) != len(variables_necesarias):
        faltantes = set(variables_necesarias) - set(variables_existentes)
        print(f"  ⚠️  Variables faltantes en {nombre_modelo}: {faltantes}")
        return None
    
    # Seleccionar datos y eliminar NaN
    df_modelo = df[variables_necesarias].dropna()
    
    if len(df_modelo) < 10:
        print(f"  ⚠️  Datos insuficientes en {nombre_modelo}: n={len(df_modelo)}")
        return None
    
    if USAR_SEMOPY:
        # Usar semopy para SEM verdadero
        try:
            # Definir modelo en sintaxis semopy
            modelo_spec = f"{outcome} ~ {' + '.join(predictores)}"
            
            modelo = Model(modelo_spec)
            modelo.fit(df_modelo)
            
            # Extraer resultados
            params = modelo.inspect()
            
            # Calcular R²
            residuos = df_modelo[outcome] - modelo.predict(df_modelo)[outcome]
            ss_res = np.sum(residuos**2)
            ss_tot = np.sum((df_modelo[outcome] - df_modelo[outcome].mean())**2)
            r_squared = 1 - (ss_res / ss_tot)
            
            # Obtener métricas de ajuste
            try:
                stats_modelo = inspect(modelo)
                aic = stats_modelo.get('AIC', np.nan)
                bic = stats_modelo.get('BIC', np.nan)
            except:
                aic = np.nan
                bic = np.nan
            
            # Formatear resultados
            resultados = {
                'Modelo': nombre_modelo,
                'Outcome': outcome,
                'n': len(df_modelo),
                'R²': r_squared,
                'AIC': aic,
                'BIC': bic,
                'Coeficientes': {}
            }
            
            # Extraer coeficientes
            for idx, row in params.iterrows():
                if row['lval'] == outcome and row['rval'] in predictores:
                    resultados['Coeficientes'][row['rval']] = {
                        'β': row['Estimate'],
                        'SE': row['Std. Err'],
                        'z': row['z-value'],
                        'p': row['p-value']
                    }
            
            return resultados
            
        except Exception as e:
            print(f"  ❌ Error en semopy para {nombre_modelo}: {e}")
            print(f"     Usando regresión OLS como respaldo...")
    
    # Fallback: Usar OLS de statsmodels
    try:
        from statsmodels.api import OLS, add_constant
        
        # Preparar datos
        X = df_modelo[predictores]
        y = df_modelo[outcome]
        
        # Agregar constante
        X_const = add_constant(X)
        
        # Ajustar modelo
        modelo_ols = OLS(y, X_const).fit()
        
        # Calcular coeficientes estandarizados
        from sklearn.preprocessing import StandardScaler
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        
        X_std = scaler_X.fit_transform(X)
        y_std = scaler_y.fit_transform(y.values.reshape(-1, 1)).flatten()
        
        modelo_std = OLS(y_std, X_std).fit()
        
        # Formatear resultados
        resultados = {
            'Modelo': nombre_modelo,
            'Outcome': outcome,
            'n': len(df_modelo),
            'R²': modelo_ols.rsquared,
            'AIC': modelo_ols.aic,
            'BIC': modelo_ols.bic,
            'Coeficientes': {}
        }
        
        # Extraer coeficientes (excluyendo constante)
        for i, pred in enumerate(predictores):
            resultados['Coeficientes'][pred] = {
                'β': modelo_ols.params[pred],
                'β_std': modelo_std.params[i],
                'SE': modelo_ols.bse[pred],
                't': modelo_ols.tvalues[pred],
                'p': modelo_ols.pvalues[pred]
            }
        
        return resultados
        
    except Exception as e:
        print(f"  ❌ Error en OLS para {nombre_modelo}: {e}")
        return None

## 4. Ejecutar Modelos para GENERALES

In [ ]:
print("="*70)
print("EJECUTANDO MODELOS SEM: GENERALES")
print("="*70)

resultados_generales = []

for outcome in Outcomes_Sumadas:
    print(f"\n📊 Modelo: {outcome}")
    print("-"*70)
    
    resultado = Ejecutar_Modelo_Path(
        df_Generales,
        outcome,
        Predictores,
        f"Generales_{outcome}"
    )
    
    if resultado:
        resultados_generales.append(resultado)
        
        print(f"  n = {resultado['n']}")
        print(f"  R² = {resultado['R²']:.4f}")
        print(f"  AIC = {resultado['AIC']:.2f}")
        print(f"  BIC = {resultado['BIC']:.2f}")
        print(f"\n  Coeficientes:")
        
        for pred, coefs in resultado['Coeficientes'].items():
            beta = coefs.get('β', coefs.get('β_std', 0))
            p_val = coefs.get('p', 1)
            sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
            
            print(f"    {pred:<25} β = {beta:>7.4f}  (p = {p_val:.4f}) {sig}")
        
        print(f"\n  {'✅ Modelo ajustado exitosamente' if resultado['R²'] > 0 else '⚠️  R² muy bajo'}")

print(f"\n{'-'*70}")
print(f"✅ {len(resultados_generales)} modelos ejecutados para Generales")
print("="*70)

## 5. Ejecutar Modelos para BALLOTAGE

In [ ]:
print("="*70)
print("EJECUTANDO MODELOS SEM: BALLOTAGE")
print("="*70)

resultados_ballotage = []

for outcome in Outcomes_Sumadas:
    print(f"\n📊 Modelo: {outcome}")
    print("-"*70)
    
    resultado = Ejecutar_Modelo_Path(
        df_Ballotage,
        outcome,
        Predictores,
        f"Ballotage_{outcome}"
    )
    
    if resultado:
        resultados_ballotage.append(resultado)
        
        print(f"  n = {resultado['n']}")
        print(f"  R² = {resultado['R²']:.4f}")
        print(f"  AIC = {resultado['AIC']:.2f}")
        print(f"  BIC = {resultado['BIC']:.2f}")
        print(f"\n  Coeficientes:")
        
        for pred, coefs in resultado['Coeficientes'].items():
            beta = coefs.get('β', coefs.get('β_std', 0))
            p_val = coefs.get('p', 1)
            sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
            
            print(f"    {pred:<25} β = {beta:>7.4f}  (p = {p_val:.4f}) {sig}")
        
        print(f"\n  {'✅ Modelo ajustado exitosamente' if resultado['R²'] > 0 else '⚠️  R² muy bajo'}")

print(f"\n{'-'*70}")
print(f"✅ {len(resultados_ballotage)} modelos ejecutados para Ballotage")
print("="*70)

## 6. Crear Tablas de Resultados

In [ ]:
def Crear_Tabla_Resultados(lista_resultados, nombre_dataset):
    """
    Convierte lista de resultados en DataFrames para exportar.
    
    Retorna:
    --------
    df_metricas : DataFrame con R², AIC, BIC
    df_coeficientes : DataFrame con β y p-valores
    """
    
    # Tabla de métricas de ajuste
    metricas_data = []
    
    for res in lista_resultados:
        metricas_data.append({
            'Outcome': res['Outcome'],
            'n': res['n'],
            'R²': res['R²'],
            'AIC': res['AIC'],
            'BIC': res['BIC']
        })
    
    df_metricas = pd.DataFrame(metricas_data)
    
    # Tabla de coeficientes
    coef_data = []
    
    for res in lista_resultados:
        for pred, coefs in res['Coeficientes'].items():
            beta = coefs.get('β', coefs.get('β_std', np.nan))
            p_val = coefs.get('p', np.nan)
            sig = '***' if p_val < 0.001 else '**' if p_val < 0.01 else '*' if p_val < 0.05 else 'ns'
            
            coef_data.append({
                'Outcome': res['Outcome'],
                'Predictor': pred,
                'β': beta,
                'SE': coefs.get('SE', np.nan),
                'p-valor': p_val,
                'Sig': sig
            })
    
    df_coeficientes = pd.DataFrame(coef_data)
    
    return df_metricas, df_coeficientes

In [ ]:
# Crear tablas para Generales
df_metricas_gen, df_coef_gen = Crear_Tabla_Resultados(resultados_generales, 'Generales')

print("\n📋 TABLA DE MÉTRICAS - GENERALES:")
print(df_metricas_gen.to_string(index=False))

print("\n📋 TABLA DE COEFICIENTES - GENERALES:")
print(df_coef_gen.to_string(index=False))

In [ ]:
# Crear tablas para Ballotage
df_metricas_bal, df_coef_bal = Crear_Tabla_Resultados(resultados_ballotage, 'Ballotage')

print("\n📋 TABLA DE MÉTRICAS - BALLOTAGE:")
print(df_metricas_bal.to_string(index=False))

print("\n📋 TABLA DE COEFICIENTES - BALLOTAGE:")
print(df_coef_bal.to_string(index=False))

## 7. Guardar Resultados en Excel

In [ ]:
# Crear carpeta de resultados
Carpeta_Resultados = os.path.join(os.getcwd(), '..', 'Data', 'Resultados_SEM')
if not os.path.exists(Carpeta_Resultados):
    os.makedirs(Carpeta_Resultados)

# Guardar Generales
archivo_gen = os.path.join(Carpeta_Resultados, 'SEM_Variables_Sumadas_Generales.xlsx')
with pd.ExcelWriter(archivo_gen, engine='openpyxl') as writer:
    df_metricas_gen.to_excel(writer, sheet_name='Métricas de Ajuste', index=False)
    df_coef_gen.to_excel(writer, sheet_name='Coeficientes', index=False)

print(f"✓ Resultados Generales guardados: SEM_Variables_Sumadas_Generales.xlsx")

# Guardar Ballotage
archivo_bal = os.path.join(Carpeta_Resultados, 'SEM_Variables_Sumadas_Ballotage.xlsx')
with pd.ExcelWriter(archivo_bal, engine='openpyxl') as writer:
    df_metricas_bal.to_excel(writer, sheet_name='Métricas de Ajuste', index=False)
    df_coef_bal.to_excel(writer, sheet_name='Coeficientes', index=False)

print(f"✓ Resultados Ballotage guardados: SEM_Variables_Sumadas_Ballotage.xlsx")

## 8. Análisis Comparativo entre Elecciones

In [ ]:
print("="*70)
print("ANÁLISIS COMPARATIVO: GENERALES vs BALLOTAGE")
print("="*70)

# Comparar R²
print("\n📊 Comparación de R² (Varianza Explicada):")
print("-"*70)

comparacion_r2 = pd.merge(
    df_metricas_gen[['Outcome', 'R²']],
    df_metricas_bal[['Outcome', 'R²']],
    on='Outcome',
    suffixes=('_Gen', '_Bal')
)
comparacion_r2['Diferencia'] = comparacion_r2['R²_Bal'] - comparacion_r2['R²_Gen']
comparacion_r2 = comparacion_r2.sort_values('R²_Gen', ascending=False)

print(comparacion_r2.to_string(index=False))

# Comparar coeficientes de Progresismo
print("\n📊 Comparación de β (Indice_Progresismo):")
print("-"*70)

coef_prog_gen = df_coef_gen[df_coef_gen['Predictor'] == 'Indice_Progresismo'][['Outcome', 'β', 'p-valor', 'Sig']]
coef_prog_bal = df_coef_bal[df_coef_bal['Predictor'] == 'Indice_Progresismo'][['Outcome', 'β', 'p-valor', 'Sig']]

comparacion_prog = pd.merge(
    coef_prog_gen,
    coef_prog_bal,
    on='Outcome',
    suffixes=('_Gen', '_Bal')
)

print(comparacion_prog.to_string(index=False))

# Comparar coeficientes de Conservadurismo
print("\n📊 Comparación de β (Indice_Conservadurismo):")
print("-"*70)

coef_cons_gen = df_coef_gen[df_coef_gen['Predictor'] == 'Indice_Conservadurismo'][['Outcome', 'β', 'p-valor', 'Sig']]
coef_cons_bal = df_coef_bal[df_coef_bal['Predictor'] == 'Indice_Conservadurismo'][['Outcome', 'β', 'p-valor', 'Sig']]

comparacion_cons = pd.merge(
    coef_cons_gen,
    coef_cons_bal,
    on='Outcome',
    suffixes=('_Gen', '_Bal')
)

print(comparacion_cons.to_string(index=False))

print("\n" + "="*70)

## 9. Identificar Mejores Modelos

In [ ]:
print("="*70)
print("MEJORES MODELOS (por R²)")
print("="*70)

print("\n🏆 Top 3 Modelos - GENERALES:")
print("-"*70)
top3_gen = df_metricas_gen.sort_values('R²', ascending=False).head(3)
print(top3_gen[['Outcome', 'R²', 'AIC', 'BIC']].to_string(index=False))

print("\n🏆 Top 3 Modelos - BALLOTAGE:")
print("-"*70)
top3_bal = df_metricas_bal.sort_values('R²', ascending=False).head(3)
print(top3_bal[['Outcome', 'R²', 'AIC', 'BIC']].to_string(index=False))

print("\n" + "="*70)

## 10. Resumen Final

In [ ]:
print("="*70)
print("RESUMEN FINAL: MODELOS SEM - VARIABLES SUMADAS")
print("="*70)

print("\n📊 Modelos ejecutados:")
print(f"  - Generales: {len(resultados_generales)} modelos")
print(f"  - Ballotage: {len(resultados_ballotage)} modelos")
print(f"  - Total: {len(resultados_generales) + len(resultados_ballotage)} modelos")

print("\n📈 Estadísticas generales:")
print(f"  Generales - R² promedio: {df_metricas_gen['R²'].mean():.4f}")
print(f"  Ballotage - R² promedio: {df_metricas_bal['R²'].mean():.4f}")

# Contar coeficientes significativos
sig_gen = len(df_coef_gen[df_coef_gen['Sig'] != 'ns'])
sig_bal = len(df_coef_bal[df_coef_bal['Sig'] != 'ns'])

print(f"\n  Generales - Coeficientes significativos: {sig_gen}/{len(df_coef_gen)}")
print(f"  Ballotage - Coeficientes significativos: {sig_bal}/{len(df_coef_bal)}")

print("\n📁 Archivos generados:")
print("  - SEM_Variables_Sumadas_Generales.xlsx")
print("  - SEM_Variables_Sumadas_Ballotage.xlsx")

print("\n🎯 Interpretación:")
print("  - R² indica qué % de varianza en el outcome explican los índices")
print("  - β positivo: a mayor índice, mayor outcome")
print("  - β negativo: a mayor índice, menor outcome")
print("  - Significancia: *** p<0.001, ** p<0.01, * p<0.05")

print("\n" + "="*70)
print("✅ ANÁLISIS SEM COMPLETADO")
print("="*70)